In [1]:
!pip install ultralytics gradio openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [2]:
from ultralytics import YOLO
import cv2
from PIL import Image
import numpy as np

model = YOLO("yolov8n-seg.pt")  # Lightweight segmentation model

def analyze_rooftop(image):
    results = model(image)
    result_img = results[0].plot()  # Image with segmentation overlay
    return Image.fromarray(result_img)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.74M/6.74M [00:00<00:00, 143MB/s]


In [3]:
def estimate_solar_potential(area_m2):
    panel_efficiency = 0.18
    irradiance = 5.5  # kWh/m²/day
    panel_power = panel_efficiency * irradiance * area_m2  # daily output in kWh
    annual_energy = panel_power * 365  # annual output

    cost_per_watt = 50  # ₹/W
    total_power_kw = annual_energy / 365 / 5  # approximate kW system size
    installation_cost = total_power_kw * 1000 * cost_per_watt

    savings_per_year = annual_energy * 8  # ₹8 per kWh
    roi_years = installation_cost / savings_per_year

    return {
        "Usable Area (m²)": round(area_m2, 2),
        "Estimated System Size (kW)": round(total_power_kw, 2),
        "Annual Output (kWh)": round(annual_energy, 2),
        "Estimated Installation Cost (₹)": round(installation_cost, 2),
        "Estimated Savings/Year (₹)": round(savings_per_year, 2),
        "Estimated Payback Period (years)": round(roi_years, 2)
    }

In [4]:
def mock_solar_analysis(image, area_estimate_m2):
    result = analyze_rooftop(image)
    roi = estimate_solar_potential(area_estimate_m2)
    return result, roi

In [5]:
import  gradio as gr
gr.Interface(
    fn=mock_solar_analysis,
    inputs=[
        gr.Image(type="filepath", label="Upload Rooftop Image"),
        gr.Slider(10, 200, step=5, label="Estimated Rooftop Area (m²)")
    ],
    outputs=[
        gr.Image(label="Rooftop Detection"),
        gr.JSON(label="Solar Analysis Report")
    ],
    title="AI-Powered Rooftop Solar Analyzer"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1279e4b8f7fb379b63.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
def get_rooftop_area_from_mask(results, pixels_per_meter=50):
    """
    Estimate rooftop area from segmentation mask.
    pixels_per_meter: assumed scale, 50px ~ 1 meter (adjust as needed)
    """
    mask = results[0].masks.data[0].cpu().numpy()  # get the first mask
    area_pixels = np.sum(mask)
    area_m2 = area_pixels / (pixels_per_meter ** 2)
    return area_m2

In [7]:
def full_rooftop_analysis(image):
    results = model(image)
    result_img = results[0].plot()

    try:
        area_m2 = get_rooftop_area_from_mask(results)
    except Exception as e:
        return Image.fromarray(result_img), {"error": f"Rooftop not detected or mask failed: {e}"}

    report = estimate_solar_potential(area_m2)
    return Image.fromarray(result_img), report

In [8]:
gr.Interface(
    fn=full_rooftop_analysis,
    inputs=gr.Image(type="filepath", label="Upload Rooftop Image"),
    outputs=[
        gr.Image(label="Rooftop Detection"),
        gr.JSON(label="Solar Analysis Report")
    ],
    title="AI-Powered Rooftop Solar Analyzer (Auto Area Detection)"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0b79f4932d32c285b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
!pip install transformers accelerate --quiet

In [10]:
from transformers import pipeline

generator = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
def offline_llm_summary(roi_dict):
    prompt = f"""
You are a smart assistant for solar energy advice.

A user has uploaded a satellite image of their rooftop. Based on the analysis, generate a short, clear, and friendly report describing the rooftop's solar installation potential. Include insights on the system size, energy output, installation cost, savings, and the return on investment.

Use this data:
- Rooftop Area: {roi_dict['Usable Area (m²)']} m²
- Estimated System Size: {roi_dict['Estimated System Size (kW)']} kW
- Annual Output: {roi_dict['Annual Output (kWh)']} kWh
- Installation Cost: ₹{roi_dict['Estimated Installation Cost (₹)']}
- Yearly Savings: ₹{roi_dict['Estimated Savings/Year (₹)']}
- Payback Period: {roi_dict['Estimated Payback Period (years)']} years

Keep it under 100 words and encourage solar adoption if feasible.
Response:
"""
    output = generator(prompt, max_new_tokens=120, temperature=0.7)[0]["generated_text"]
    return output.split("Response:")[-1].strip()

In [12]:
def full_rooftop_analysis_with_local_llm(image):
    results = model(image)
    result_img = results[0].plot()

    try:
        area_m2 = get_rooftop_area_from_mask(results)
        report = estimate_solar_potential(area_m2)
        summary = offline_llm_summary(report)
        return Image.fromarray(result_img), report, summary
    except Exception as e:
        return Image.fromarray(result_img), {"error": str(e)}, "LLM summary failed."

In [13]:
gr.Interface(
    fn=full_rooftop_analysis_with_local_llm,
    inputs=gr.Image(type="filepath", label="Upload Rooftop Image"),
    outputs=[
        gr.Image(label="Rooftop Detection"),
        gr.JSON(label="Solar Analysis Report"),
        gr.Textbox(label="LLM Summary (Offline)")
    ],
    title="AI Rooftop Solar Analyzer (Offline LLM)"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b41df9ffab63a9b246.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib --quiet

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-4_8j3056
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-4_8j3056
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=490df91846ca36d66302a55c56bd9809517dafac6f229373b47944ed2f93f043
  Stored in directory: /tmp/pip-ephem-wheel-cache-nidt17ky/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything


In [15]:
import os

# Create directory and download the model
os.makedirs("sam_weights", exist_ok=True)

!wget -O sam_weights/sam_vit_h.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2025-05-27 10:04:12--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_weights/sam_vit_h.pth’

sam_weights/sam_vit 100%[===================>]   2.39G   191MB/s    in 21s     

2025-05-27 10:04:33 (116 MB/s) - ‘sam_weights/sam_vit_h.pth’ saved [2564550879/2564550879]



In [16]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load SAM
sam = sam_model_registry["vit_h"](checkpoint="sam_weights/sam_vit_h.pth")
sam.to("cuda")

# Mask generator
mask_generator = SamAutomaticMaskGenerator(sam)

In [17]:
def segment_rooftop_with_sam(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    masks = mask_generator.generate(image_rgb)

    # Combine all SAM masks into one binary mask
    combined_mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for mask in masks:
        combined_mask = np.logical_or(combined_mask, mask["segmentation"])

    combined_mask = combined_mask.astype(np.uint8)

    # Create colored overlay on the image
    overlay = image_rgb.copy()
    overlay[combined_mask == 1] = [0, 255, 0]  # green mask for rooftop

    # Blend original + mask for visibility
    alpha = 0.5
    blended = cv2.addWeighted(image_rgb, 1 - alpha, overlay, alpha, 0)

    # Return visual + mask area (pixel count)
    return Image.fromarray(blended), np.sum(combined_mask)

In [18]:
def area_from_sam_mask(pixel_count, pixels_per_meter=50):
    return pixel_count / (pixels_per_meter ** 2)

In [19]:
def full_rooftop_analysis_with_sam(image_path):
    masked_image, pixel_count = segment_rooftop_with_sam(image_path)
    area_m2 = area_from_sam_mask(pixel_count)
    report = estimate_solar_potential(area_m2)
    summary = offline_llm_summary(report)

    return masked_image, report, summary

In [20]:
gr.Interface(
    fn=full_rooftop_analysis_with_sam,
    inputs=gr.Image(type="filepath", label="Upload Satellite Rooftop Image"),
    outputs=[
        gr.Image(label="Rooftop Segmentation (SAM)"),
        gr.JSON(label="Solar Analysis Report"),
        gr.Textbox(label="LLM Summary (Offline)")
    ],
    title="SAM-Powered Rooftop Solar Analyzer"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c2bf9ef10d42fd352a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [21]:
def segment_rooftop_with_clean_overlay(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    masks = mask_generator.generate(image_rgb)

    # Filter + sort largest masks
    masks = sorted(masks, key=lambda x: np.sum(x["segmentation"]), reverse=True)
    masks = [m for m in masks if np.sum(m["segmentation"]) > 500][:30]  # max 30 big masks

    annotated_img = image_rgb.copy()
    total_px = 0

    for mask in masks:
        seg = mask['segmentation'].astype(np.uint8)
        area_px = np.sum(seg)
        total_px += area_px

        overlay = np.zeros_like(image_rgb)
        overlay[seg == 1] = (0, 255, 255)

        annotated_img = cv2.addWeighted(annotated_img, 1, overlay, 0.4, 0)

        x, y, w, h = cv2.boundingRect(seg)
        area_m2 = area_px / (50 ** 2)
        cv2.rectangle(annotated_img, (x, y), (x + w, y + h), (255, 255, 255), 1)
        cv2.putText(annotated_img, f"{area_m2:.2f} m²", (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

    return Image.fromarray(annotated_img), total_px

In [22]:
def full_rooftop_analysis_with_visual_overlay(image_path):
    annotated_img, total_px = segment_rooftop_with_clean_overlay(image_path)
    area_m2 = area_from_sam_mask(total_px)
    report = estimate_solar_potential(area_m2)
    summary = offline_llm_summary(report)

    return annotated_img, report, summary

In [26]:
demo=gr.Interface(
    fn=full_rooftop_analysis_with_visual_overlay,
    inputs=gr.Image(type="filepath", label="Upload Satellite Image"),
    outputs=[
        gr.Image(label="Rooftop Detection + Area"),
        gr.JSON(label="Solar Report"),
        gr.Textbox(label="LLM Summary")
    ],
    title="Solar Analyzer with Visual Area Overlay"
)
if __name__ == "__main__":
    demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09d74577c05eea7d7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [24]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.7 MB/s eta 0:00:00


In [25]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas

def generate_simple_pdf(report_data, summary_text, output_path="solar_report_simple.pdf"):
    c = canvas.Canvas(output_path, pagesize=A4)
    width, height = A4
    x, y = 50, height - 50

    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(x, y, "🌞 Rooftop Solar Installation Report")
    y -= 40

    # Solar Report Data
    c.setFont("Helvetica", 12)
    for key, value in report_data.items():
        c.drawString(x, y, f"{key}: {value}")
        y -= 20

    # AI Summary
    y -= 20
    c.setFont("Helvetica-Bold", 13)
    c.drawString(x, y, "🧠 AI Summary:")
    y -= 20

    c.setFont("Helvetica-Oblique", 11)
    for line in summary_text.split('\n'):
        c.drawString(x, y, line.strip())
        y -= 15

    c.setFont("Helvetica", 9)
    y = 40
    c.drawString(x, y, "Generated by Devraj Singh – Internship Project, 2025")
    c.save()

# Sample data (use your actual results)
report_dict = {
    "Usable Area (m²)": "43.36",
    "Estimated System Size (kW)": "8.58",
    "Annual Output (kWh)": "15667.56",
    "Installation Cost (₹)": "429248.16",
    "Estimated Savings/Year (₹)": "125340.46",
    "Payback Period (years)": "3.42"
}

llm_summary = (
    "Your rooftop can support an 8.58 kW solar system, producing approx. 15,667 kWh annually.\n"
    "With a cost of ₹4.29L, you'll save over ₹1.25L every year.\n"
    "The system pays for itself in just 3.4 years—a smart green investment!"
)

generate_simple_pdf(report_dict, llm_summary)